<a href="https://colab.research.google.com/github/leochartrand/IFT615/blob/main/CNN/CNN2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Cette démonstration est inspirée du cours "CS50’s Introduction to Artificial Intelligence with Python" donné à l'Université d'Harvard.*

https://cs50.harvard.edu/ai/2024/

https://creativecommons.org/licenses/by-nc-sa/4.0/





In [8]:
import numpy as np
from PIL import Image, ImageFilter
from skimage.io import imshow
import matplotlib.pyplot as plt
from IPython.display import display
import torch.nn as nn
import torchvision

In [7]:
train_data = torchvision.datasets.MNIST(root = './', train = True,
                        transform = torchvision.transforms.ToTensor(), download = True)

test_data = torchvision.datasets.MNIST(root = './', train = False,
                                       transform = torchvision.transforms.ToTensor())

100%|██████████| 9912422/9912422 [00:00<00:00, 102049825.59it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 10996341.12it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 30427743.52it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 5217893.39it/s]


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [9]:
model = nn.Sequential(

    # Couche convolutive. Apprend 32 filtres en utilisant un noyau 3x3
    nn.Conv2d(1, 32, (3, 3)),

    # Activation ReLU
    nn.ReLU(),

    # Couche de max-pooling de taille 2x2
    nn.MaxPool2d((2, 2)),

    # On applatie les données
    nn.Flatten(),

    # Couche dense
    nn.Linear(5408, 128),
    nn.ReLU(),

    # Couche dense finale avec comme sortie les probabilitées pour les 10 chiffres
    nn.Linear(128, 10),
    nn.Softmax(dim=1),
)